# Trading Code Generator

This notebook creates a code generator that produces trading code to buy and sell equities in a simulated environment based on free APIs. It uses Gradio for the UI, similar to the approach in day5.ipynb.


In [14]:
import os
import io
import sys
import time
import random
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import display
from huggingface_hub import InferenceClient


In [15]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
hf_token = os.getenv('HF_TOKEN')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if hf_token:
    print(f"Hugging Face Token exists and begins {hf_token[:8]}")
else:
    print("Hugging Face Token not set")


OpenAI API Key exists and begins sk-proj-
Hugging Face Token exists and begins hf_fNncb


In [16]:
openai_client = OpenAI()
hf_client = InferenceClient(token=hf_token)


In [46]:
models = ["gpt-4o", "gpt-3.5-turbo", "meta-llama/Llama-2-70b-chat-hf"]

def generate_with_openai(model, messages):
    response = openai_client.chat.completions.create(
        model=model, 
        messages=messages
    )
    return response.choices[0].message.content

def generate_with_hf(model, messages):
    prompt = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "system":
            prompt += f"<s>[INST] {content} [/INST]</s>\n"
        elif role == "user":
            prompt += f"<s>[INST] {content} [/INST]</s>\n"
        else:
            prompt += f"{content}\n"
    
    response = hf_client.text_generation(
        prompt,
        model=model,
        max_new_tokens=1024,
        temperature=0.7,
        repetition_penalty=1.2
    )
    return response


In [47]:
CSS = """
:root {
  --py-color: #209dd7;
  --trading-color: #27ae60;
  --accent:   #753991;
  --card:     #161a22;
  --text:     #e9eef5;
}

/* Full-width layout */
.gradio-container {
  max-width: 100% !important;
  padding: 0 40px !important;
}

/* Code card styling */
.card {
  background: var(--card);
  border: 1px solid rgba(255,255,255,.08);
  border-radius: 14px;
  padding: 10px;
}

/* Make code block scrollable but fixed height */
#code-block {
  max-height: 400px !important;
  overflow-y: auto !important;
}

#code-block .cm-editor {
  height: 400px !important;
}

/* Buttons */
.generate-btn button {
  background: var(--accent) !important;
  border-color: rgba(255,255,255,.12) !important;
  color: white !important;
  font-weight: 700;
}
.run-btn button {
  background: #202631 !important;
  color: var(--text) !important;
  border-color: rgba(255,255,255,.12) !important;
}
.run-btn.py button:hover { box-shadow: 0 0 0 2px var(--py-color) inset; }
.run-btn.trading button:hover { box-shadow: 0 0 0 2px var(--trading-color) inset; }
.generate-btn button:hover { box-shadow: 0 0 0 2px var(--accent) inset; }

/* Outputs with color tint */
.py-out textarea {
  background: linear-gradient(180deg, rgba(32,157,215,.18), rgba(32,157,215,.10));
  border: 1px solid rgba(32,157,215,.35) !important;
  color: rgba(32,157,215,1) !important;
  font-weight: 600;
}
.trading-out textarea {
  background: linear-gradient(180deg, rgba(39,174,96,.18), rgba(39,174,96,.10));
  border: 1px solid rgba(39,174,96,.35) !important;
  color: rgba(39,174,96,1) !important;
  font-weight: 600;
}

/* Align controls neatly */
.controls .wrap {
  gap: 10px;
  justify-content: center;
  align-items: center;
}
"""


In [ ]:
system_prompt = """
You are an expert algorithmic trading code generator. Generate clean, bug-free Python code for trading strategies.

Generate code that:
1. Uses synthetic data generation only - no API calls
2. Implements the specified trading strategy
3. Uses proper error handling
4. Visualizes strategy performance with buy/sell signals
5. Calculates performance metrics
6. Handles edge cases properly

REQUIREMENTS:
1. Include if __name__ == "__main__": block that executes immediately
2. Define all variables before use
3. Pass parameters between functions, avoid global variables
4. NO explanatory text outside of code
5. NO markdown blocks or language indicators
6. Code must execute without user input
7. Use str() for pandas objects in f-strings
8. Use .copy() for DataFrame views that will be modified
9. Include min_periods in rolling calculations
10. Check array lengths before scatter plots
11. Configure logging properly
12. Include helper functions for formatting and plotting

Respond ONLY with Python code. No explanations or markdown.
"""

def user_prompt_for(description):
    return f"""
Generate Python code for a trading strategy:

{description}

Requirements:
1. Use synthetic data generation only
2. Implement the strategy exactly as described
3. Include backtesting functionality
4. Visualize results with matplotlib
5. Calculate performance metrics
6. Handle all edge cases
7. No comments needed

Make the code complete and runnable as-is with all necessary imports.
"""


In [87]:
def messages_for(description):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(description)}
    ]


In [ ]:
def validate_code(code):
    issues = []
    if "import yfinance" not in code and "from yfinance" not in code:
        issues.append("Missing yfinance import")
    if "import matplotlib" not in code and "from matplotlib" not in code:
        issues.append("Missing matplotlib import")
    if "__name__ == \"__main__\"" not in code and "__name__ == '__main__'" not in code:
        issues.append("Missing if __name__ == '__main__' block")
    if "f\"" in code or "f'" in code:
        lines = code.split('\n')
        for i, line in enumerate(lines):
            if ('f"' in line or "f'" in line) and ('data[' in line or '.iloc' in line or '.loc' in line):
                issues.append(f"Potentially unsafe f-string formatting with pandas objects on line {i+1}")
    if "try:" in code and "except" not in code:
        issues.append("Try block without except clause")
    if "rolling" in code and "min_periods" not in code:
        issues.append("Rolling window without min_periods parameter (may produce NaN values)")
    if ".loc" in code and "iloc" not in code and "copy()" not in code:
        issues.append("Potential pandas SettingWithCopyWarning - consider using .copy() before modifications")
    lines = code.split('\n')
    defined_vars = set()
    for line in lines:
        if line.strip().startswith('#') or not line.strip():
            continue
        if '=' in line and not line.strip().startswith('if') and not line.strip().startswith('elif') and not line.strip().startswith('while'):
            var_name = line.split('=')[0].strip()
            if var_name:
                defined_vars.add(var_name)
    if issues:
        return False, issues
    return True, []

def generate_trading_code(model, description, force_gpt4=False):
    messages = messages_for(description)
    if force_gpt4:
        try:
            reply = generate_with_openai("gpt-4o", messages)
        except Exception as e:
            print(f"Error using GPT-4o: {e}. Falling back to selected model.")
            if "gpt" in model.lower():
                reply = generate_with_openai(model, messages)
            else:
                reply = generate_with_hf(model, messages)
    else:
        if "gpt" in model.lower():
            reply = generate_with_openai(model, messages)
        else:
            reply = generate_with_hf(model, messages)
    reply = reply.replace('```python','').replace('```','')
    is_valid, issues = validate_code(reply)
    max_attempts = 3
    attempt = 0
    fix_model = "gpt-4o" if force_gpt4 else model
    while not is_valid and attempt < max_attempts and ("gpt" in model.lower() or force_gpt4):
        attempt += 1
        fix_messages = messages.copy()
        fix_messages.append({"role": "assistant", "content": reply})
        fix_request = f"""The code has the following issues that need to be fixed:
{chr(10).join([f"- {issue}" for issue in issues])}

Please provide a completely corrected version that addresses these issues. Make sure to:

1. Avoid using f-strings with pandas Series or DataFrame objects directly
2. Always handle NaN values in calculations with proper checks
3. Use proper error handling with try/except blocks around all API calls and calculations
4. Include min_periods parameter in rolling window calculations
5. Use .copy() when creating views of DataFrames that will be modified
6. Make sure all variables are properly defined before use
7. Add yfinance timeout settings: yf.set_timeout(30)
8. Add proper logging for all steps
9. Use synthetic data generation as a fallback if API calls fail
10. Include proper if __name__ == "__main__" block

Return ONLY the corrected code with no explanation or markdown formatting.
"""
        fix_messages.append({"role": "user", "content": fix_request})
        try:
            if force_gpt4:
                fixed_reply = generate_with_openai("gpt-4o", fix_messages)
            else:
                if "gpt" in model.lower():
                    fixed_reply = generate_with_openai(model, fix_messages)
                else:
                    fixed_reply = generate_with_hf(model, fix_messages)
            fixed_reply = fixed_reply.replace('```python','').replace('```','')
            is_fixed_valid, fixed_issues = validate_code(fixed_reply)
            if is_fixed_valid or len(fixed_issues) < len(issues):
                reply = fixed_reply
                is_valid = is_fixed_valid
                issues = fixed_issues
        except Exception as e:
            print(f"Error during fix attempt {attempt}: {e}")
    reply = add_safety_features(reply)
    return reply

def add_safety_features(code):
    if "pandas" in code:
        safety_imports = """
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)

def safe_format(obj):
    if isinstance(obj, (pd.Series, pd.DataFrame)):
        return str(obj)
    return obj
"""
        import_lines = [i for i, line in enumerate(code.split('\n')) if 'import' in line]
        if import_lines:
            lines = code.split('\n')
            lines.insert(import_lines[-1] + 1, safety_imports)
            code = '\n'.join(lines)
    code = code.replace("yf.set_timeout(30)", "")
    code = code.replace("yf.pdr_override()", "")
    lines = code.split('\n')
    for i, line in enumerate(lines):
        if 'f"' in line or "f'" in line:
            if any(term in line for term in ['data[', '.iloc', '.loc', 'Series', 'DataFrame']):
                for term in ['.mean()', '.sum()', '.std()', '.min()', '.max()']:
                    if term in line:
                        lines[i] = line.replace(f"{term}", f"{term})")
                        lines[i] = lines[i].replace("f\"", "f\"{safe_format(")
                        lines[i] = lines[i].replace("f'", "f'{safe_format(")
    code = '\n'.join(lines)
    if "plt.scatter" in code or ".scatter" in code:
        scatter_safety = """
def safe_scatter(ax, x, y, *args, **kwargs):
    if len(x) != len(y):
        min_len = min(len(x), len(y))
        x = x[:min_len]
        y = y[:min_len]
    if len(x) == 0 or len(y) == 0:
        return None
    return ax.scatter(x, y, *args, **kwargs)
"""
        func_lines = [i for i, line in enumerate(code.split('\n')) if line.startswith('def ')]
        if func_lines:
            lines = code.split('\n')
            lines.insert(func_lines[0], scatter_safety)
            code = '\n'.join(lines)
            code = code.replace("plt.scatter(", "safe_scatter(plt.gca(), ")
            code = code.replace(".scatter(", "safe_scatter(")
    if "yfinance" in code and "generate_synthetic_data" not in code:
        synthetic_data_func = """
def generate_synthetic_data(ticker='AAPL', start_date=None, end_date=None, days=252, seed=42):
    import numpy as np
    import pandas as pd
    from datetime import datetime, timedelta
    if start_date is None:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
    elif end_date is None:
        if isinstance(start_date, str):
            start_date = pd.to_datetime(start_date)
        end_date = datetime.now()
    np.random.seed(seed)
    if isinstance(start_date, str):
        start = pd.to_datetime(start_date)
    else:
        start = start_date
    if isinstance(end_date, str):
        end = pd.to_datetime(end_date)
    else:
        end = end_date
    days = (end - start).days + 1
    price = 100
    prices = [price]
    for _ in range(days):
        change = np.random.normal(0, 0.01)
        price *= (1 + change)
        prices.append(price)
    dates = pd.date_range(start=start, end=end, periods=len(prices))
    df = pd.DataFrame({
        'Open': prices[:-1],
        'High': [p * 1.01 for p in prices[:-1]],
        'Low': [p * 0.99 for p in prices[:-1]],
        'Close': prices[1:],
        'Volume': [np.random.randint(1000000, 10000000) for _ in range(len(prices)-1)]
    }, index=dates[:-1])
    return df
"""
        func_lines = [i for i, line in enumerate(code.split('\n')) if line.startswith('def ')]
        if func_lines:
            lines = code.split('\n')
            lines.insert(func_lines[0], synthetic_data_func)
            code = '\n'.join(lines)
    if "logging" in code and "basicConfig" not in code:
        logging_config = """
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)
"""
        import_lines = [i for i, line in enumerate(code.split('\n')) if 'import' in line]
        if import_lines:
            lines = code.split('\n')
            lines.insert(import_lines[-1] + 1, logging_config)
            code = '\n'.join(lines)
    if "yfinance" in code and "try:" not in code:
        lines = code.split('\n')
        for i, line in enumerate(lines):
            if "yf.download" in line and "try:" not in lines[max(0, i-5):i]:
                indent = len(line) - len(line.lstrip())
                indent_str = " " * indent
                lines[i] = f"{indent_str}try:\n{indent_str}    {line}\n{indent_str}except Exception as e:\n{indent_str}    logging.error(f\"Error fetching data: {{e}}\")\n{indent_str}    # Use synthetic data as fallback\n{indent_str}    data = generate_synthetic_data(ticker, start_date, end_date)"
                code = '\n'.join(lines)
                break
    if "synthetic data" in code.lower() and "yf.download" in code:
        lines = code.split('\n')
        for i, line in enumerate(lines):
            if "yf.download" in line:
                indent = len(line) - len(line.lstrip())
                indent_str = " " * indent
                comment = f"{indent_str}# Using synthetic data instead of API call\n"
                synthetic = f"{indent_str}data = generate_synthetic_data(ticker, start_date, end_date)\n"
                lines[i] = f"{indent_str}# {line.strip()} # Commented out to avoid API issues"
                lines.insert(i+1, comment + synthetic)
                code = '\n'.join(lines)
                break
    if "plt.figure" in code:
        lines = code.split('\n')
        for i, line in enumerate(lines):
            if "plt.figure" in line and "try:" not in lines[max(0, i-5):i]:
                indent = len(line) - len(line.lstrip())
                indent_str = " " * indent
                try_line = f"{indent_str}try:\n{indent_str}    "
                except_line = f"\n{indent_str}except Exception as e:\n{indent_str}    logging.error(f\"Error in plotting: {{e}}\")"
                j = i
                while j < len(lines) and (j == i or lines[j].startswith(indent_str)):
                    j += 1
                for k in range(i, j):
                    if lines[k].strip():
                        lines[k] = indent_str + "    " + lines[k].lstrip()
                lines.insert(i, try_line.rstrip())
                lines.insert(j+1, except_line)
                code = '\n'.join(lines)
                break
    lines = code.split('\n')
    for i, line in enumerate(lines):
        if "print(" in line and any(term in line for term in ['data[', '.iloc', '.loc', 'Series', 'DataFrame']):
            lines[i] = line.replace("print(", "print(safe_format(")
            if "))" not in lines[i] and ")," in lines[i]:
                lines[i] = lines[i].replace("),", ")),", 1)
            elif "))" not in lines[i] and ")" in lines[i]:
                lines[i] = lines[i].replace(")", "))", 1)
    code = '\n'.join(lines)
    return code


In [114]:
def run_python(code):
    # Create a completely separate namespace for execution
    namespace = {
        '__name__': '__main__',
        '__builtins__': __builtins__
    }
    
    # Modify the code to use a non-interactive matplotlib backend
    # and fix pandas formatting issues
    modified_code = """
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend

# Import yfinance without setting timeout (not available in all versions)
import yfinance as yf

# Configure logging to show in the output
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)

# Fix pandas formatting issues
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)

# Override print to ensure it flushes immediately
import builtins
original_print = builtins.print
def custom_print(*args, **kwargs):
    result = original_print(*args, **kwargs)
    import sys
    sys.stdout.flush()
    return result
builtins.print = custom_print

# Helper function to safely format pandas objects
def safe_format(obj):
    if isinstance(obj, (pd.Series, pd.DataFrame)):
        return str(obj)
    else:
        return obj
"""
    
    # Add the user's code
    modified_code += "\n" + code
    
    # Capture all output
    output_buffer = io.StringIO()
    
    # Save original stdout and redirect to our buffer
    original_stdout = sys.stdout
    sys.stdout = output_buffer
    
    # Add timestamp for execution start
    print(f"[{time.strftime('%H:%M:%S')}] Executing code...")
    
    try:
        # Execute the modified code
        exec(modified_code, namespace)
        print(f"\n[{time.strftime('%H:%M:%S')}] Execution completed successfully.")
        
    except ModuleNotFoundError as e:
        missing_module = str(e).split("'")[1]
        print(f"\nError: Missing module '{missing_module}'. Click 'Install Dependencies' to install it.")
        namespace["__missing_module__"] = missing_module
        
    except Exception as e:
        print(f"\n[{time.strftime('%H:%M:%S')}] Error during execution: {str(e)}")
        import traceback
        print(traceback.format_exc())
        
    finally:
        # Restore original stdout
        sys.stdout = original_stdout
        
    # Return the captured output
    return output_buffer.getvalue()

def install_dependencies(code):
    import re
    import subprocess
    
    import_pattern = r'(?:from|import)\s+([a-zA-Z0-9_]+)(?:\s+(?:import|as))?'
    imports = re.findall(import_pattern, code)
    
    std_libs = ['os', 'sys', 'io', 'time', 'datetime', 'random', 'math', 're', 'json', 
                'collections', 'itertools', 'functools', 'operator', 'pathlib', 'typing']
    
    modules_to_install = [module for module in imports if module not in std_libs]
    
    if not modules_to_install:
        return "No external dependencies found to install."
    
    results = []
    for module in modules_to_install:
        try:
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", module],
                capture_output=True,
                text=True,
                check=False
            )
            
            if result.returncode == 0:
                results.append(f"Successfully installed {module}")
            else:
                results.append(f"Failed to install {module}: {result.stderr}")
        except Exception as e:
            results.append(f"Error installing {module}: {str(e)}")
    
    return "\n".join(results)


In [109]:
trading_strategies = [
    {
        "name": "Moving Average Crossover",
        "description": "Moving Average Crossover strategy for S&P 500 stocks. Buy when the 20-day moving average crosses above the 50-day moving average, and sell when it crosses below.",
        "buy_signal": "20-day MA crosses above 50-day MA",
        "sell_signal": "20-day MA crosses below 50-day MA",
        "timeframe": "Daily",
        "risk_level": "Medium"
    },
    {
        "name": "RSI Mean Reversion",
        "description": "Mean reversion strategy that buys stocks when RSI falls below 30 (oversold) and sells when RSI rises above 70 (overbought).",
        "buy_signal": "RSI below 30 (oversold)",
        "sell_signal": "RSI above 70 (overbought)",
        "timeframe": "Daily",
        "risk_level": "Medium"
    },
    {
        "name": "Momentum Strategy",
        "description": "Momentum strategy that buys the top 5 performing stocks from the Dow Jones Industrial Average over the past month and rebalances monthly.",
        "buy_signal": "Stock in top 5 performers over past month",
        "sell_signal": "Stock no longer in top 5 performers at rebalance",
        "timeframe": "Monthly",
        "risk_level": "High"
    },
    {
        "name": "Pairs Trading",
        "description": "Pairs trading strategy that identifies correlated stock pairs and trades on the divergence and convergence of their price relationship.",
        "buy_signal": "Pairs ratio deviates 2+ standard deviations below mean",
        "sell_signal": "Pairs ratio returns to mean or exceeds mean",
        "timeframe": "Daily",
        "risk_level": "Medium-High"
    },
    {
        "name": "Bollinger Band Breakout",
        "description": "Volatility breakout strategy that buys when a stock breaks out of its upper Bollinger Band and sells when it reverts to the mean.",
        "buy_signal": "Price breaks above upper Bollinger Band (2 std dev)",
        "sell_signal": "Price reverts to middle Bollinger Band (SMA)",
        "timeframe": "Daily",
        "risk_level": "High"
    },
    {
        "name": "MACD Crossover",
        "description": "MACD crossover strategy that buys when the MACD line crosses above the signal line and sells when it crosses below.",
        "buy_signal": "MACD line crosses above signal line",
        "sell_signal": "MACD line crosses below signal line",
        "timeframe": "Daily",
        "risk_level": "Medium"
    },
    {
        "name": "Golden Cross",
        "description": "Golden Cross strategy that buys when the 50-day moving average crosses above the 200-day moving average and sells on the Death Cross (opposite).",
        "buy_signal": "50-day MA crosses above 200-day MA",
        "sell_signal": "50-day MA crosses below 200-day MA",
        "timeframe": "Daily",
        "risk_level": "Low"
    }
]

sample_strategies = [strategy["description"] for strategy in trading_strategies]


In [110]:
default_description = """
Create a moving average crossover strategy with the following specifications:
- Use yfinance to download historical data for a list of stocks (AAPL, MSFT, AMZN, GOOGL, META)
- Calculate 20-day and 50-day moving averages
- Generate buy signals when the 20-day MA crosses above the 50-day MA
- Generate sell signals when the 20-day MA crosses below the 50-day MA
- Implement a simple backtesting framework to evaluate the strategy
- Calculate performance metrics: total return, annualized return, Sharpe ratio, max drawdown
- Visualize the equity curve, buy/sell signals, and moving averages
"""


In [ ]:
with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title="Trading Code Generator") as ui:
    with gr.Row():
        gr.HTML("<h1 style='text-align: center; margin-bottom: 0.5rem;'>Trading Strategy Code Generator</h1>")
    
    with gr.Row():
        # Left column - Controls
        with gr.Column(scale=1):
            strategy_dropdown = gr.Dropdown(
                label="Select Trading Strategy",
                choices=[strategy["name"] for strategy in trading_strategies],
                value=trading_strategies[0]["name"]
            )
            
            with gr.Accordion("Strategy Details", open=False):
                strategy_info = gr.JSON(
                    value=trading_strategies[0]
                )
            
            model = gr.Dropdown(
                label="Select Model",
                choices=models,
                value=models[0]
            )
            
            description = gr.TextArea(
                label="Strategy Description (Edit to customize)",
                value=trading_strategies[0]["description"],
                lines=4
            )
            
            with gr.Row():
                generate = gr.Button("Generate Code", variant="primary", size="sm")
                run = gr.Button("Run Code", size="sm")
                install_deps = gr.Button("Install Dependencies", size="sm")
        
        # Right column - Code and Output
        with gr.Column(scale=2):
            trading_code = gr.Code(
                label="Generated Trading Code",
                value="",
                language="python",
                lines=20,
                elem_id="code-block",
                show_label=True
            )
                
            output = gr.TextArea(
                label="Execution Output",
                lines=8,
                elem_classes=["trading-out"]
            )
    
    def update_strategy_info(strategy_name):
        selected = next((s for s in trading_strategies if s["name"] == strategy_name), None)
        if selected:
            return selected, selected["description"]
        return trading_strategies[0], trading_strategies[0]["description"]
    
    strategy_dropdown.change(
        fn=update_strategy_info,
        inputs=strategy_dropdown,
        outputs=[strategy_info, description]
    )
    
    # Function to show validation results when generating code
    def generate_with_validation(model, description):
        # Always use GPT-4o for better code quality
        code = generate_trading_code(model, description, force_gpt4=True)
        is_valid, issues = validate_code(code)
        
        validation_message = ""
        if is_valid:
            validation_message = "Code validation passed ✓"
        else:
            validation_message = "Code validation warnings:\n" + "\n".join([f"- {issue}" for issue in issues])
        
        return code, validation_message
    
    generate.click(
        fn=generate_with_validation,
        inputs=[model, description],
        outputs=[trading_code, output]
    )
    
    run.click(
        fn=run_python,
        inputs=[trading_code],
        outputs=[output]
    )
    
    install_deps.click(
        fn=install_dependencies,
        inputs=[trading_code],
        outputs=[output]
    )

ui.launch(inbrowser=True)


2025-10-22 18:30:21,233 - INFO - HTTP Request: GET http://127.0.0.1:7875/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-10-22 18:30:21,238 - INFO - HTTP Request: HEAD http://127.0.0.1:7875/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


2025-10-22 18:30:24,092 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-10-22 18:31:03,437 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-22 18:31:15,743 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-22 18:31:28,425 - ERROR - Error fetching data: periods must be a number, got 2025-10-22 18:31:28.425210
2025-10-22 18:31:28,429 - INFO - Synthetic data generated for tickers: AAPL
2025-10-22 18:31:28,432 - INFO - Moving averages calculated with windows 20 and 50
2025-10-22 18:31:28,434 - INFO - Signals generated based on moving average crossover
2025-10-22 18:31:28,438 - INFO - Performance calculated
2025-10-22 18:31:28,438 - INFO - Total Return: -0.010752455100331848, Sharpe Ratio: 0.18162435507214664, Max Drawdown: -0.19919271751608258
2025-10-22 18:32:28,496 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-

## Testing the Trading Code Generator

Let's test the trading code generator with a specific strategy description and model.


In [ ]:
test_description = """
Create a simple RSI-based mean reversion strategy:
- Use AAPL stock data for the past 2 years
- Calculate the 14-day RSI indicator
- Buy when RSI falls below 30 (oversold)
- Sell when RSI rises above 70 (overbought)
- Include visualization of entry/exit points
- Calculate performance metrics
"""

test_model = "gpt-3.5-turbo"

generated_code = generate_trading_code(test_model, test_description)
print("Generated trading code:")
print(generated_code)


In [ ]:
try:
    output = run_python(generated_code)
    print(output)
except Exception as e:
    print(f"Error running the generated code: {e}")


In [ ]:
# Fixed version of the code
fixed_code = """
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_moving_averages(data, short_window=20, long_window=50):
    \"\"\"
    Calculate short and long term moving averages
    \"\"\"
    data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()
    return data

def generate_signals(data, short_window=20, long_window=50):
    \"\"\"
    Generate buy/sell signals based on moving average crossover strategy
    \"\"\"
    data['Signal'] = 0
    data['Signal'][short_window:] = np.where(
        data['Short_MA'][short_window:] > data['Long_MA'][short_window:], 1, -1)
    data['Position'] = data['Signal'].shift(1)
    data['Position'].fillna(0, inplace=True)  # Fill NaN values with 0
    return data

def backtest_strategy(data):
    \"\"\"
    Backtest the trading strategy and calculate performance metrics
    \"\"\"
    data['Returns'] = data['Close'].pct_change()
    data['Strategy_Returns'] = data['Returns'] * data['Position']
    
    # Replace NaN values with 0
    data['Strategy_Returns'].fillna(0, inplace=True)

    cumulative_returns = (1 + data['Strategy_Returns']).cumprod()
    
    # Calculate metrics
    total_return = cumulative_returns.iloc[-1] - 1
    sharpe_ratio = np.sqrt(252) * (data['Strategy_Returns'].mean() / data['Strategy_Returns'].std())
    max_drawdown = ((cumulative_returns / cumulative_returns.cummax()) - 1).min()

    metrics = {
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }

    return cumulative_returns, metrics

def plot_results(data, cumulative_returns, ticker):
    \"\"\"
    Plot the performance of the trading strategy
    \"\"\"
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})
    
    # Price and MA plot
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.plot(data.index, data['Short_MA'], label='20-day MA', alpha=0.7)
    ax1.plot(data.index, data['Long_MA'], label='50-day MA', alpha=0.7)
    
    # Add buy/sell signals
    buy_signals = data[data['Signal'] > data['Signal'].shift(1)]
    sell_signals = data[data['Signal'] < data['Signal'].shift(1)]
    
    ax1.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', s=100, label='Buy Signal')
    ax1.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', s=100, label='Sell Signal')
    
    ax1.set_title(f'Moving Average Crossover Strategy on {ticker}')
    ax1.set_ylabel('Price ($)')
    ax1.legend(loc='best')
    ax1.grid(True)
    
    # Returns plot
    ax2.plot(cumulative_returns.index, cumulative_returns, label='Cumulative Strategy Returns', color='blue')
    ax2.set_title('Cumulative Returns')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Returns')
    ax2.legend(loc='best')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

if __name__ == \"__main__\":
    # User inputs
    ticker = 'SPY'  # Example: S&P 500 ETF
    start_date = (datetime.now() - timedelta(days=365*2)).strftime('%Y-%m-%d')
    end_date = datetime.now().strftime('%Y-%m-%d')

    # Strategy parameters
    short_window = 20
    long_window = 50

    # Fetch data
    try:
        logging.info(f\"Fetching data for {ticker} from {start_date} to {end_date}...\")
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        logging.info(f\"Data fetched successfully. Got {len(stock_data)} data points.\")
    except Exception as e:
        logging.error(f\"Failed to fetch data: {e}\")
        raise SystemExit(e)

    try:
        # Preprocess and generate signals
        stock_data = calculate_moving_averages(stock_data, short_window, long_window)
        stock_data = generate_signals(stock_data, short_window, long_window)

        # Backtest the strategy
        cumulative_returns, metrics = backtest_strategy(stock_data)

        # Display metrics
        for key, value in metrics.items():
            logging.info(f\"{key}: {value:.4f}\")

        # Plot results
        plot_results(stock_data, cumulative_returns, ticker)
    except Exception as e:
        logging.error(f\"Error while executing strategy: {e}\")
"""

# Display the fixed code
print("Fixed code:")
print(fixed_code)


In [ ]:
# Run the fixed code
output = run_python(fixed_code)
print(output)


In [ ]:
# Let's also update our system_prompt to ensure the generated code works properly

system_prompt = """
You are an expert algorithmic trading code generator. Your task is to generate Python code for trading strategies based on user requirements.
The code should be well-structured, efficient, and ready to run in a simulated environment.

The generated code should:
1. Use the yfinance library for fetching stock data
2. Implement the specified trading strategy
3. Include proper error handling and logging
4. Include visualization of the strategy performance with clear buy/sell signals
5. Calculate and display relevant metrics (returns, Sharpe ratio, drawdown, etc.)
6. Handle NaN values and edge cases properly
7. Include informative print statements or logging to show progress

IMPORTANT: Make sure all variables are properly defined before use, especially in functions.
Always pass necessary parameters between functions rather than relying on global variables.

Respond only with Python code. Do not provide any explanation other than occasional comments in the code.
"""


In [ ]:
# Let's test the updated system prompt with a simple strategy

test_description_2 = """
Create a simple Bollinger Bands strategy:
- Use AAPL stock data for the past 1 year
- Calculate Bollinger Bands with 20-day SMA and 2 standard deviations
- Buy when price touches the lower band
- Sell when price touches the upper band
- Include visualization of entry/exit points
- Calculate performance metrics
"""

test_model = "gpt-3.5-turbo"
generated_code_2 = generate_trading_code(test_model, test_description_2)
print("Generated trading code with updated prompt:")
print(generated_code_2)


In [ ]:
# Let's test the run function with live logging

test_code = """
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Define ticker and date range
ticker = 'AAPL'
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download data
print(f"Downloading data for {ticker}...")
data = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {len(data)} rows of data")

# Calculate RSI
print("Calculating RSI...")
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# Generate signals
print("Generating trading signals...")
data['Signal'] = 0
data.loc[data['RSI'] < 30, 'Signal'] = 1  # Buy signal
data.loc[data['RSI'] > 70, 'Signal'] = -1  # Sell signal

# Count signals
buy_signals = len(data[data['Signal'] == 1])
sell_signals = len(data[data['Signal'] == -1])
print(f"Generated {buy_signals} buy signals and {sell_signals} sell signals")

# Print sample of the data
print("\\nSample of the processed data:")
print(data[['Close', 'RSI', 'Signal']].tail())

print("\\nAnalysis complete!")
"""

output = run_python(test_code)
print(output)


In [ ]:
# Let's test the improved run function again

test_code_2 = """
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Define ticker and date range
ticker = 'AAPL'
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download data
print(f"Downloading data for {ticker}...")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)
print(f"Downloaded {len(data)} rows of data")

# Calculate RSI
print("Calculating RSI...")
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# Generate signals
print("Generating trading signals...")
data['Signal'] = 0
data.loc[data['RSI'] < 30, 'Signal'] = 1  # Buy signal
data.loc[data['RSI'] > 70, 'Signal'] = -1  # Sell signal

# Count signals
buy_signals = len(data[data['Signal'] == 1])
sell_signals = len(data[data['Signal'] == -1])
print(f"Generated {buy_signals} buy signals and {sell_signals} sell signals")

# Print sample of the data
print("\\nSample of the processed data:")
print(data[['Close', 'RSI', 'Signal']].tail())

print("\\nAnalysis complete!")
"""

output = run_python(test_code_2)
print(output)


In [ ]:
# Test the completely rewritten run function

simple_test = """
print("Hello from the trading code generator!")
print("Testing output capture...")

# Simulate some data processing
import numpy as np
data = np.random.rand(5, 3)
print("Generated random data:")
print(data)

# Show a calculation
result = np.mean(data, axis=0)
print("Mean of each column:")
print(result)

print("Test complete!")
"""

output = run_python(simple_test)
print("Output from execution:")
print(output)


In [ ]:
# Test the improved code generation with a strategy that typically causes formatting issues

test_description_3 = """
Create a simple trading strategy that:
- Uses AAPL stock data for the past year
- Calculates both RSI and Bollinger Bands
- Buys when price is below lower Bollinger Band AND RSI is below 30
- Sells when price is above upper Bollinger Band OR RSI is above 70
- Includes proper error handling for all calculations
- Visualizes the entry/exit points and performance
"""

test_model = "gpt-3.5-turbo"
generated_code_3 = generate_trading_code(test_model, test_description_3)
print("Generated trading code with enhanced validation:")
print(generated_code_3)


In [ ]:
# Let's test running the generated code

output = run_python(generated_code_3)
print("Execution output:")
print(output)


In [ ]:
# Test the improved run function with a simple example that prints output

simple_test_2 = """
print("This is a test of the output capture system")
print("Line 1 of output")
print("Line 2 of output")

# Import and use numpy
import numpy as np
data = np.random.rand(3, 3)
print("Random matrix:")
print(data)

# Create a simple plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.plot([1, 2, 3, 4], [10, 20, 25, 30], 'ro-')
plt.title('Simple Plot')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.grid(True)
plt.savefig('simple_plot.png')  # Save instead of showing
print("Plot saved to simple_plot.png")

print("Test complete!")
"""

output = run_python(simple_test_2)
print("Output from execution:")
print(output)


In [ ]:
# Test with a simpler example that won't time out

simple_test_3 = """
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Generate synthetic stock data
def generate_stock_data(days=252, volatility=0.01):
    logging.info(f"Generating {days} days of synthetic stock data")
    np.random.seed(42)
    price = 100
    prices = [price]
    
    for _ in range(days - 1):
        change = np.random.normal(0, volatility)
        price *= (1 + change)
        prices.append(price)
    
    dates = pd.date_range(end=pd.Timestamp.today(), periods=days)
    df = pd.DataFrame({
        'Close': prices,
        'Open': [p * (1 - volatility/2) for p in prices],
        'High': [p * (1 + volatility) for p in prices],
        'Low': [p * (1 - volatility) for p in prices],
        'Volume': [np.random.randint(100000, 10000000) for _ in range(days)]
    }, index=dates)
    
    logging.info(f"Generated data with shape {df.shape}")
    return df

# Calculate RSI
def calculate_rsi(data, window=14):
    logging.info(f"Calculating RSI with {window}-day window")
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=window, min_periods=1).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window, min_periods=1).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Main function
if __name__ == "__main__":
    # Generate data
    data = generate_stock_data()
    
    # Calculate RSI
    data['RSI'] = calculate_rsi(data)
    
    # Generate signals
    logging.info("Generating trading signals")
    data['Signal'] = 0
    data.loc[data['RSI'] < 30, 'Signal'] = 1  # Buy signal
    data.loc[data['RSI'] > 70, 'Signal'] = -1  # Sell signal
    
    # Count signals
    buy_signals = len(data[data['Signal'] == 1])
    sell_signals = len(data[data['Signal'] == -1])
    logging.info(f"Generated {buy_signals} buy signals and {sell_signals} sell signals")
    
    # Plot the results
    logging.info("Creating visualization")
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [3, 1]})
    
    # Price chart
    ax1.plot(data.index, data['Close'], label='Close Price')
    
    # Add buy/sell signals
    buy_points = data[data['Signal'] == 1]
    sell_points = data[data['Signal'] == -1]
    
    ax1.scatter(buy_points.index, buy_points['Close'], marker='^', color='g', s=100, label='Buy')
    ax1.scatter(sell_points.index, sell_points['Close'], marker='v', color='r', s=100, label='Sell')
    
    ax1.set_title('Stock Price with RSI Signals')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True)
    
    # RSI chart
    ax2.plot(data.index, data['RSI'], color='purple', label='RSI')
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax2.set_title('RSI Indicator')
    ax2.set_ylabel('RSI')
    ax2.set_ylim(0, 100)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('rsi_strategy.png')
    logging.info("Plot saved to rsi_strategy.png")
    
    # Print sample of data
    logging.info("Sample of the processed data:")
    print(data[['Close', 'RSI', 'Signal']].tail())
    
    logging.info("Analysis complete!")
"""

output = run_python(simple_test_3)
print("Output from execution:")
print(output)


In [ ]:
# Test the enhanced code generation with GPT-4o

test_description_4 = """
Create a trading strategy that:
- Uses both MACD and RSI indicators
- Buys when MACD crosses above signal line AND RSI is below 40
- Sells when MACD crosses below signal line OR RSI is above 70
- Includes proper visualization with buy/sell signals
- Uses synthetic data if API calls fail
- Calculates performance metrics including Sharpe ratio and max drawdown
"""

print("Generating trading code with GPT-4o...")
generated_code_4 = generate_trading_code("gpt-4o", test_description_4, force_gpt4=True)
print("Code generation complete. Validating...")
is_valid, issues = validate_code(generated_code_4)

if issues:
    print(f"Validation found {len(issues)} issues:")
    for issue in issues:
        print(f"- {issue}")
else:
    print("Code validation passed ✓")

print("\nGenerated code snippet (first 20 lines):")
print("\n".join(generated_code_4.split("\n")[:20]))


In [ ]:
# Let's run the generated code to test it

output = run_python(generated_code_4)
print("Execution output:")
print(output)


In [ ]:
# Let's test again with the fixed timeout setting

test_description_5 = """
Create a simple trading strategy that:
- Uses synthetic data generation to avoid API timeouts
- Implements a simple moving average crossover (5-day and 20-day)
- Includes proper visualization with buy/sell signals
- Calculates basic performance metrics
"""

print("Generating trading code with proper yfinance timeout settings...")
generated_code_5 = generate_trading_code("gpt-4o", test_description_5, force_gpt4=True)
print("Code generation complete.")

# Run the generated code
output = run_python(generated_code_5)
print("Execution output:")
print(output)


In [ ]:
# Test with a simpler strategy that focuses on scatter plot safety

test_description_6 = """
Create a simple trading strategy that:
- Uses synthetic data generation only (no API calls)
- Implements a simple RSI-based strategy (buy when RSI < 30, sell when RSI > 70)
- Includes visualization with buy/sell signals using scatter plots
- Calculates basic performance metrics
- Uses proper error handling for all operations
"""

print("Generating trading code with scatter plot safety...")
generated_code_6 = generate_trading_code("gpt-4o", test_description_6, force_gpt4=True)
print("Code generation complete.")

# Run the generated code
output = run_python(generated_code_6)
print("Execution output:")
print(output)


In [ ]:
# Test with a fixed example that properly handles pandas formatting and scatter plots

test_fixed_code = """
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)

# Helper function for safe formatting of pandas objects
def safe_format(obj):
    if isinstance(obj, (pd.Series, pd.DataFrame)):
        return str(obj)
    return obj

# Helper function to safely create scatter plots
def safe_scatter(ax, x, y, *args, **kwargs):
    # Ensure x and y are the same length
    if len(x) != len(y):
        logging.warning(f"Scatter plot inputs have different lengths: x={len(x)}, y={len(y)}")
        # Find the minimum length
        min_len = min(len(x), len(y))
        x = x[:min_len]
        y = y[:min_len]
    
    # Check for empty arrays
    if len(x) == 0 or len(y) == 0:
        logging.warning("Empty arrays passed to scatter plot, skipping")
        return None
    
    return ax.scatter(x, y, *args, **kwargs)

# Generate synthetic data
def generate_synthetic_data(ticker='AAPL', days=252, seed=42):
    logging.info(f"Generating synthetic data for {ticker}")
    np.random.seed(seed)
    
    # Generate price data
    price = 100  # Starting price
    prices = [price]
    
    for _ in range(days):
        change = np.random.normal(0, 0.01)  # 1% volatility
        price *= (1 + change)
        prices.append(price)
    
    # Create date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    dates = pd.date_range(start=start_date, end=end_date, periods=len(prices))
    
    # Create DataFrame
    df = pd.DataFrame({
        'Open': prices[:-1],
        'High': [p * 1.01 for p in prices[:-1]],
        'Low': [p * 0.99 for p in prices[:-1]],
        'Close': prices[1:],
        'Volume': [np.random.randint(1000000, 10000000) for _ in range(len(prices)-1)]
    }, index=dates[:-1])
    
    logging.info(f"Generated {len(df)} days of data for {ticker}")
    return df

# Calculate RSI
def calculate_rsi(data, window=14):
    logging.info(f"Calculating RSI with {window}-day window")
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Generate signals
def generate_signals(data):
    logging.info("Generating trading signals")
    data['Signal'] = 0
    data.loc[data['RSI'] < 30, 'Signal'] = 1  # Buy signal
    data.loc[data['RSI'] > 70, 'Signal'] = -1  # Sell signal
    
    # Count signals
    buy_signals = len(data[data['Signal'] == 1])
    sell_signals = len(data[data['Signal'] == -1])
    logging.info(f"Generated {buy_signals} buy signals and {sell_signals} sell signals")
    return data

# Backtest strategy
def backtest_strategy(data):
    logging.info("Backtesting strategy")
    data['Returns'] = data['Close'].pct_change()
    data['Strategy'] = data['Signal'].shift(1) * data['Returns']
    
    # Replace NaN values
    data['Strategy'].fillna(0, inplace=True)
    
    # Calculate cumulative returns
    data['Cumulative'] = (1 + data['Strategy']).cumprod()
    
    # Calculate metrics
    total_return = data['Cumulative'].iloc[-1] - 1
    sharpe = np.sqrt(252) * data['Strategy'].mean() / data['Strategy'].std()
    max_dd = (data['Cumulative'] / data['Cumulative'].cummax() - 1).min()
    
    logging.info(f"Total Return: {total_return:.4f}")
    logging.info(f"Sharpe Ratio: {sharpe:.4f}")
    logging.info(f"Max Drawdown: {max_dd:.4f}")
    
    return data

# Visualize results
def visualize_results(data, ticker):
    logging.info("Creating visualization")
    try:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [3, 1]})
        
        # Price chart
        ax1.plot(data.index, data['Close'], label='Close Price')
        
        # Add buy/sell signals
        buy_points = data[data['Signal'] == 1]
        sell_points = data[data['Signal'] == -1]
        
        # Use safe scatter to avoid "x and y must be the same size" error
        if not buy_points.empty:
            safe_scatter(ax1, buy_points.index, buy_points['Close'], marker='^', color='g', s=100, label='Buy')
        
        if not sell_points.empty:
            safe_scatter(ax1, sell_points.index, sell_points['Close'], marker='v', color='r', s=100, label='Sell')
        
        ax1.set_title(f'RSI Strategy for {ticker}')
        ax1.set_ylabel('Price')
        ax1.legend()
        ax1.grid(True)
        
        # RSI chart
        ax2.plot(data.index, data['RSI'], color='purple', label='RSI')
        ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
        ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
        ax2.set_title('RSI Indicator')
        ax2.set_ylabel('RSI')
        ax2.set_ylim(0, 100)
        ax2.grid(True)
        
        plt.tight_layout()
        plt.savefig('rsi_strategy.png')
        logging.info("Plot saved to rsi_strategy.png")
    except Exception as e:
        logging.error(f"Error in visualization: {e}")

if __name__ == "__main__":
    # Settings
    ticker = 'AAPL'
    days = 252  # One year of trading days
    
    # Generate data
    data = generate_synthetic_data(ticker, days)
    
    # Calculate RSI
    data['RSI'] = calculate_rsi(data)
    
    # Generate signals
    data = generate_signals(data)
    
    # Backtest strategy
    data = backtest_strategy(data)
    
    # Visualize results
    visualize_results(data, ticker)
    
    # Print sample of data
    logging.info("Sample of the processed data:")
    print(data[['Close', 'RSI', 'Signal', 'Strategy', 'Cumulative']].tail())
    
    logging.info("Analysis complete!")
"""

output = run_python(test_fixed_code)
print("Output from execution:")
print(output)


In [ ]:
# Test the fix for indentation error in plotting code

test_code_with_plotting = """
import matplotlib.pyplot as plt
import numpy as np
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)

# Simple plotting function
def create_plot():
    # Generate some data
    x = np.linspace(0, 10, 100)
    y = np.sin(x)
    
    # Create plot
    logging.info("Creating sine wave plot")
    plt.figure(figsize=(10, 6))
    plt.plot(x, y)
    plt.title('Sine Wave')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.savefig('sine_wave.png')
    logging.info("Plot saved to sine_wave.png")

if __name__ == "__main__":
    create_plot()
"""

# Apply safety features to the code
enhanced_code = add_safety_features(test_code_with_plotting)
print("Code with safety features applied:")
print(enhanced_code)

# Run the enhanced code
output = run_python(enhanced_code)
print("\nExecution output:")
print(output)
